# Baseline

## Imports

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
!pip install sentence-transformers
!pip install rdflib
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

! pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.5 MB/s 
     |████████████████████████████████| 5.8 MB 33.6 MB/s 
     |████████████████████████████████| 1.3 MB 53.2 MB/s 
     |████████████████████████████████| 182 kB 54.9 MB/s 
     |████████████████████████████████| 7.6 MB 67.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=9e4d803e00300b7ef57bb493c5bb8bf6f02ddeecc0b7aa4eba602ba9905c49c0
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 500 kB 4.9 MB/s 
     |████████████████████████████████| 41 kB 667 kB/s 
1.13.0+cu116
     |████████████████████████████████| 9.4 MB 4.5 MB/s 
  

In [3]:
from torch_geometric.nn import GATConv, GCNConv, RGATConv


In [4]:
import torch.nn.functional as F
import os
import torch
import numpy as np
import random


from typing import List, Callable

from torch import Tensor, device
import pandas as pd

In [5]:
os.chdir("/content/gdrive/My Drive/1_Studium/2_Master/Master Thesis/2_Projekt/Scratch/all_in/MAGNET")

In [6]:
import model_functions.helper_funcs as helper_funcs
import model_functions.model_definitions as model_definition
import model_functions.losses as losses_funcs

Data

In [7]:
os.chdir("/content/gdrive/My Drive/1_Studium/2_Master/Master Thesis/2_Projekt/Scratch/all_in/data/dbp15k/")

In [8]:
baseline_data = torch.load("dbp15k.pt")

In [9]:
from torch_geometric.data import Data

In [10]:
baseline_data_new = Data(x_one=baseline_data.x_one, edge_index_one = baseline_data.edge_index_one,x_two=baseline_data.x_two,
     edge_index_two = baseline_data.edge_index_two, train_set_left=baseline_data.test_set_left,
     train_set_right=baseline_data.test_set_right, test_set_left = baseline_data.train_set_left,
    test_set_right=baseline_data.train_set_right)

In [11]:
baseline_data_new

Data(x_one=[19993, 512], edge_index_one=[2, 115722], x_two=[19661, 512], edge_index_two=[2, 105998], train_set_left=[10500], train_set_right=[10500], test_set_left=[4499], test_set_right=[4499])

Model

HIER AUFPASSEN, DASS ES RICHTIG ANGEPASST WIRD!!!!

In [28]:
PATH = "/content/gdrive/My Drive/1_Studium/2_Master/Master Thesis/2_Projekt/Scratch/all_in/MAGNET/results/trained_models/dbp15k__GCNConv_V11_800-900"
loaded_model = model_definition.GCNClassic(512)
loaded_model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

Other adjustments

In [29]:
device = 'cpu' 
source_data = baseline_data_new.to(device) 
#model = GCNClassic(512).to(device)
model = loaded_model
myloss = losses_funcs.CosineSimilarityLoss4(model)


optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=0.0005, momentum=0.9)

def train():
    model.train()
    optimizer.zero_grad()
    
    lefts, rights = losses_funcs.create_anti_alignment_v41(epoch, matches, source_data)
    loss = myloss(source_data, lefts, rights)
    loss.backward()
    optimizer.step()
    return float(loss)

In [30]:
data = baseline_data_new

In [31]:
precisions = []
recalls = []
losses = []

precision_tests = []
recall_tests = []

wrong_sets_left_train = []
wrong_sets_right_train = []
matches = []
all_matches = []

for epoch in range(1, 100):
    loss = train()
    prec, rec, matches = helper_funcs.test_harder('train', model, source_data)
    prec_test, rec_test, matches_test = helper_funcs.test_harder('test', model, source_data)
    print(f'TRAINING Epoch: {epoch:02d}, Loss: {loss:.4f}, Precision: {prec:.4f} , Recall: {rec:.4f} ')
    print(f'TEST Epoch: {epoch:02d}, Precision: {prec_test:.4f} , Recall: {rec_test:.4f} ')


    precisions.append(prec)
    recalls.append(rec)
    losses.append(loss)
    precision_tests.append(prec_test)
    recall_tests.append(rec_test)
    x, y, wrong_sets_left, wrong_sets_right = helper_funcs.filter_out_wrong_matches(matches, data)
    wrong_sets_left_train.append(wrong_sets_left)
    wrong_sets_right_train.append(wrong_sets_right)
    all_matches.append(matches)

TRAINING Epoch: 01, Loss: 0.1830, Precision: 0.2545 , Recall: 0.3005 
TEST Epoch: 01, Precision: 0.2086 , Recall: 0.2847 
TRAINING Epoch: 02, Loss: 0.1844, Precision: 0.2550 , Recall: 0.3009 
TEST Epoch: 02, Precision: 0.2090 , Recall: 0.2852 
TRAINING Epoch: 03, Loss: 0.1828, Precision: 0.2549 , Recall: 0.3008 
TEST Epoch: 03, Precision: 0.2097 , Recall: 0.2861 
TRAINING Epoch: 04, Loss: 0.1840, Precision: 0.2549 , Recall: 0.3009 
TEST Epoch: 04, Precision: 0.2095 , Recall: 0.2852 
TRAINING Epoch: 05, Loss: 0.1831, Precision: 0.2553 , Recall: 0.3009 
TEST Epoch: 05, Precision: 0.2105 , Recall: 0.2863 
TRAINING Epoch: 06, Loss: 0.1832, Precision: 0.2546 , Recall: 0.2999 
TEST Epoch: 06, Precision: 0.2115 , Recall: 0.2870 
TRAINING Epoch: 07, Loss: 0.1842, Precision: 0.2546 , Recall: 0.2998 
TEST Epoch: 07, Precision: 0.2101 , Recall: 0.2856 
TRAINING Epoch: 08, Loss: 0.1828, Precision: 0.2551 , Recall: 0.3000 
TEST Epoch: 08, Precision: 0.2098 , Recall: 0.2854 
TRAINING Epoch: 09, Loss

Save output

In [32]:
import pickle
version = 'V11_900-1000'
a_or_b = "baseline"
with open("/content/gdrive/My Drive/1_Studium/2_Master/Master Thesis/2_Projekt/Scratch/all_in/MAGNET/results/trainings/GCN/wrong_sets_left_" + version + "_"  , "wb") as fp:   #Pickling
        pickle.dump(wrong_sets_left_train, fp)

In [33]:
import pandas as pd

In [34]:
with open("/content/gdrive/My Drive/1_Studium/2_Master/Master Thesis/2_Projekt/Scratch/all_in/MAGNET/results/trainings/GCN/wrong_sets_right_" + version + "_" ,"wb") as fp:   #Pickling
        pickle.dump(wrong_sets_right_train, fp)

In [35]:
with open("/content/gdrive/My Drive/1_Studium/2_Master/Master Thesis/2_Projekt/Scratch/all_in/MAGNET/results/trainings/GCN/matches" + version + "_", "wb") as fp:   #Pickling
        pickle.dump(all_matches, fp)

In [36]:
model_name = 'GCNConv'
df = pd.DataFrame(list(zip(losses, precisions, recalls, precision_tests, recall_tests)), columns =['losses', 'precisions_train', 'recalls_train', 'precisions_test', 'recalls_test'])

In [37]:
model_name = 'GCNConv'


In [38]:
track = "dbp15k"

In [39]:
path = "/content/gdrive/My Drive/1_Studium/2_Master/Master Thesis/2_Projekt/Scratch/all_in/MAGNET/results/trainings/GCN/" + track + "_" + "_" + model_name + "_" + version
df.to_csv(path + ".csv")

In [40]:
path = "/content/gdrive/My Drive/1_Studium/2_Master/Master Thesis/2_Projekt/Scratch/all_in/MAGNET/results/trained_models/" + track + "_" + "_" + model_name + "_" + version
torch.save(model.state_dict(), path)